In [0]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need this cell
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow1/free_chat/chinese/main')

In [0]:
%tensorflow_version 1.x

In [3]:
import tensorflow as tf
import time

from pathlib import Path

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 1.15.0
GPU Enabled: False


In [0]:
def get_vocab(f_path):
  word2idx = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip('\n')
      word2idx[line] = i
  return word2idx

def parse_fn(text):
  parse = lambda text: [[params['char2idx'].get(c, len(params['char2idx'])) for c in list(text)]]
  return {'words': parse(text)}

In [5]:
params = {'export_dir': '../model/lstm_seq2seq_export'}

params['char2idx'] = get_vocab('../vocab/char.txt')

params['idx2char'] = {idx: char for char, idx in params['char2idx'].items()}

subdirs = [x for x in Path(params['export_dir']).iterdir()
           if x.is_dir() and 'temp' not in str(x)]
latest = str(sorted(subdirs)[-1])
print(latest)

predict_fn = tf.contrib.predictor.from_saved_model(latest)

../model/lstm_seq2seq_export/1575616207
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from ../model/lstm_seq2seq_export/1575616207/variables/variables


In [0]:
while True:
  text_inp = input('Input:')
  t0 = time.time()
  predictions = predict_fn(parse_fn(text_inp))
  #print('Shape:', predictions['output'].shape)
  max_len = 0
  output = None
  for j in range(3):
    indices = [params['idx2char'].get(idx, len(params['idx2char'])) for idx in predictions['output'][0, :, j] if (idx != 0 and idx != 2)]
    sent = ' '.join(indices)
    if len(indices) > max_len:
      max_len = len(indices)
      output = sent
    #print('Rank_{}:'.format(j+1), sent)
  print('Output:', output)
  print('Cost %.2f sec' % (time.time() - t0))
  print()

Input:你好
Output: 你 好 主 人 ， 今 天 愉 快 吗
Cost 0.86 sec

Input:早上好
Output: 早 上 好 O ( ∩ _ ∩ ) O
Cost 0.17 sec

Input:晚上好
Output: 晚 上 好 O ( ∩ _ ∩ ) O
Cost 0.13 sec

Input:再见
Output: 再 见 亲 爱 的 主 淫
Cost 0.17 sec

Input:好久不见
Output: 好 久 不 见 ~ ~
Cost 0.11 sec

Input:想死你了
Output: 我 也 想 死 你 了
Cost 0.11 sec

Input:谢谢你
Output: 不 谢 谢 你
Cost 0.15 sec

Input:爱你
Output: 我 也 爱 你   主 人 〜 ( ^ ○ ^ )
Cost 0.15 sec

Input:你叫什么名字
Output: 我 叫 小 樱 桃 ， 天 翼 给 我 取 的 名 字 我
Cost 0.17 sec

Input:你几岁了
Output: 我 今 年 1 岁 了 ！
Cost 0.16 sec

Input:现在几点了
Output: 我 现 在 还 想 她 多 少 点
Cost 0.16 sec

Input:今天天气怎么样
Output: 我 怎 么 了 ？
Cost 0.17 sec

Input:我们现在在哪里
Output: 必 须 的 ' ' ' ' ' ' ' ' ' ' ' '
Cost 0.17 sec

Input:你能给我讲个笑话吗
Output: 从 来 都 不 能 给 我 讲 笑 话 了
Cost 0.16 sec

Input:你是男孩还是女孩呀
Output: 男 孩 纸
Cost 0.12 sec

Input:你会几种语言呀
Output: 全 世 界 语 言 的 我 都 会
Cost 0.15 sec

Input:你能陪我玩吗
Output: 当 然 可 以 陪 你 玩 ~
Cost 0.11 sec

Input:说话可以大声一点吗
Output: 可 以 啊 。 。 。
Cost 0.14 sec

Input:天气真好
Output: 那 天 气 很 好 的 天 气 , 天 气 真 好
Cost 0.17 sec

